In [1]:
from sklearn.ensemble import AdaBoostClassifier,GradientBoostingClassifier
from sklearn.ensemble import RandomForestClassifier, VotingClassifier
import pandas as pd
import numpy as np
import os
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
import pandas as pd
import numpy as np
from sklearn.neighbors import KNeighborsClassifier, KNeighborsRegressor
from sklearn.metrics import r2_score, mean_squared_error, confusion_matrix, accuracy_score, roc_curve, roc_auc_score,log_loss,classification_report
from sklearn.model_selection import train_test_split, KFold, cross_val_score, GridSearchCV,RandomizedSearchCV
from sklearn.preprocessing import PolynomialFeatures, LabelEncoder, OneHotEncoder,MinMaxScaler, StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from sklearn.linear_model import ElasticNet, LogisticRegression, LinearRegression
from sklearn.compose import make_column_transformer, make_column_selector
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.naive_bayes import BernoulliNB, GaussianNB
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.tree import plot_tree
from sklearn.tree import DecisionTreeRegressor
import warnings

from sklearn.ensemble import BaggingClassifier,RandomForestClassifier,RandomForestRegressor
from sklearn.ensemble import BaggingRegressor
import lightgbm as lgb
from catboost import CatBoostClassifier,CatBoostRegressor
from xgboost import XGBClassifier
from sklearn.ensemble import StackingClassifier,StackingRegressor

In [2]:
df = pd.read_csv('Satellite.csv',sep=';')
df.columns

Index(['x.1', 'x.2', 'x.3', 'x.4', 'x.5', 'x.6', 'x.7', 'x.8', 'x.9', 'x.10',
       'x.11', 'x.12', 'x.13', 'x.14', 'x.15', 'x.16', 'x.17', 'x.18', 'x.19',
       'x.20', 'x.21', 'x.22', 'x.23', 'x.24', 'x.25', 'x.26', 'x.27', 'x.28',
       'x.29', 'x.30', 'x.31', 'x.32', 'x.33', 'x.34', 'x.35', 'x.36',
       'classes'],
      dtype='object')

In [3]:
y = df['classes']
X = df.drop('classes',axis=1)

In [4]:
X_train,X_test,y_train,y_test = train_test_split(X,y,random_state=24,test_size=0.3,stratify = y)

In [5]:
knn = KNeighborsClassifier()
nb = GaussianNB()
dtc = DecisionTreeClassifier(random_state=24)
svm = SVC(random_state = 24, probability=True)
lr = LogisticRegression(random_state= 24)
stack = StackingClassifier(estimators=[('KNN',knn),('NB',nb),('TREE',dtc),('SVM',svm)],
                           final_estimator=lr)


In [6]:
stack.fit(X_train,y_train)
y_pred = stack.predict(X_test)
print(accuracy_score(y_test,y_pred))

0.905748316934231


In [7]:
# y_pred_prob = stack.predict_proba(X_test)[:,1]
# roc_auc_score(y_test,y_pred_prob)

In [8]:
stack = StackingClassifier(estimators=[('KNN',knn),('NB',nb),('TREE',dtc),('SVM',svm)],
                           final_estimator=lr,passthrough = True)
stack.fit(X_train,y_train)
y_pred = stack.predict(X_test)
print(accuracy_score(y_test,y_pred))

C:\Users\dai.STUDENTSDC\AppData\Local\anaconda3\Lib\site-packages\sklearn\linear_model\_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


0.7928534438114966


In [9]:
rfc = RandomForestClassifier(random_state=24)
stack = StackingClassifier(estimators=[('KNN',knn),('NB',nb),('TREE',dtc),('SVM',svm)],
                           final_estimator=rfc,passthrough = True)
stack.fit(X_train,y_train)
y_pred = stack.predict(X_test)
print(accuracy_score(y_test,y_pred))

0.9093733816675298


In [10]:
stack.get_params()

{'cv': None,
 'estimators': [('KNN', KNeighborsClassifier()),
  ('NB', GaussianNB()),
  ('TREE', DecisionTreeClassifier(random_state=24)),
  ('SVM', SVC(probability=True, random_state=24))],
 'final_estimator__bootstrap': True,
 'final_estimator__ccp_alpha': 0.0,
 'final_estimator__class_weight': None,
 'final_estimator__criterion': 'gini',
 'final_estimator__max_depth': None,
 'final_estimator__max_features': 'sqrt',
 'final_estimator__max_leaf_nodes': None,
 'final_estimator__max_samples': None,
 'final_estimator__min_impurity_decrease': 0.0,
 'final_estimator__min_samples_leaf': 1,
 'final_estimator__min_samples_split': 2,
 'final_estimator__min_weight_fraction_leaf': 0.0,
 'final_estimator__monotonic_cst': None,
 'final_estimator__n_estimators': 100,
 'final_estimator__n_jobs': None,
 'final_estimator__oob_score': False,
 'final_estimator__random_state': 24,
 'final_estimator__verbose': 0,
 'final_estimator__warm_start': False,
 'final_estimator': RandomForestClassifier(random_stat

In [18]:
params = {
    'final_estimator__max_depth': [3,4,5],
    
    
    'TREE__max_depth': [None,2,4],
    'final_estimator__n_estimators': [10,50,100],
    'passthrough': [True,False],
    
}
kfold = StratifiedKFold(n_splits=5,shuffle=True,random_state=24)

In [20]:
stack = StackingClassifier(estimators=[('KNN',knn),('NB',nb),('TREE',dtc)],
                           final_estimator=rfc)

In [22]:
gcv = GridSearchCV(stack,param_grid=params,cv=kfold,scoring='neg_log_loss',verbose=3,n_jobs=-1)
gcv.fit(X,y)

Fitting 5 folds for each of 54 candidates, totalling 270 fits


GridSearchCV(cv=StratifiedKFold(n_splits=5, random_state=24, shuffle=True),
             estimator=StackingClassifier(estimators=[('KNN',
                                                       KNeighborsClassifier()),
                                                      ('NB', GaussianNB()),
                                                      ('TREE',
                                                       DecisionTreeClassifier(random_state=24))],
                                          final_estimator=RandomForestClassifier(random_state=24)),
             n_jobs=-1,
             param_grid={'TREE__max_depth': [None, 2, 4],
                         'final_estimator__max_depth': [3, 4, 5],
                         'final_estimator__n_estimators': [10, 50, 100],
                         'passthrough': [True, False]},
             scoring='neg_log_loss', verbose=3)

In [30]:
gcv.best_score_

-0.3334776139903915

In [24]:
model = gcv.best_estimator_

In [26]:
model

StackingClassifier(estimators=[('KNN', KNeighborsClassifier()),
                               ('NB', GaussianNB()),
                               ('TREE',
                                DecisionTreeClassifier(random_state=24))],
                   final_estimator=RandomForestClassifier(max_depth=5,
                                                          random_state=24))

In [34]:
import pickle
pkfile = open('satellite.pkl','wb')
pickle.dump(model,pkfile)